In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install librosa

In [3]:
!pip install soundfile


In [4]:
import os
import librosa
import numpy as np



In [83]:
import os
import pandas as pd

def read_emotion_dataset(dataset_folder):
    data = {'file_path': [], 'emotion': []}

    for emotion in os.listdir(dataset_folder):
        emotion_folder = os.path.join(dataset_folder, emotion)

        if os.path.isdir(emotion_folder):

            for file_name in os.listdir(emotion_folder):
                file_path = os.path.join(emotion_folder, file_name)

                if os.path.isfile(file_path):

                    data['file_path'].append(file_path)
                    data['emotion'].append(emotion)

    emotion_dataset = pd.DataFrame(data)
    return emotion_dataset

dataset_folder = "/content/drive/MyDrive/train_emotion"
emotion_dataset = read_emotion_dataset(dataset_folder)

print(emotion_dataset.tail(50))


                                             file_path   emotion
544  /content/drive/MyDrive/train_emotion/surprise/...  surprise
545  /content/drive/MyDrive/train_emotion/surprise/...  surprise
546  /content/drive/MyDrive/train_emotion/surprise/...  surprise
547  /content/drive/MyDrive/train_emotion/surprise/...  surprise
548  /content/drive/MyDrive/train_emotion/surprise/...  surprise
549  /content/drive/MyDrive/train_emotion/surprise/...  surprise
550  /content/drive/MyDrive/train_emotion/surprise/...  surprise
551  /content/drive/MyDrive/train_emotion/surprise/...  surprise
552  /content/drive/MyDrive/train_emotion/surprise/...  surprise
553  /content/drive/MyDrive/train_emotion/surprise/...  surprise
554  /content/drive/MyDrive/train_emotion/surprise/...  surprise
555  /content/drive/MyDrive/train_emotion/surprise/...  surprise
556  /content/drive/MyDrive/train_emotion/surprise/...  surprise
557  /content/drive/MyDrive/train_emotion/surprise/...  surprise
558  /content/drive/MyDri

In [6]:
!pip install soundfile sounddevice


In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
emotion_df=emotion_dataset

In [23]:
!pip install librosa
!pip install resampy

In [27]:
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder

def extract_features(file_path):
    audio, _ = librosa.load(file_path, sr=None)
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=_ , n_mfcc=13), axis=1)
    return mfccs
X = [extract_features(file_path) for file_path in emotion_df['file_path']]
X = np.vstack(X)
y = emotion_df['emotion']

le = LabelEncoder()
y = le.fit_transform(y)


In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

In [86]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(397, 13)
(397,)
(197, 13)
(197,)


In [87]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [88]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=300, random_state=42)

In [89]:
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 98.48%


In [90]:
import pickle
train_test_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test
}

# Save the data to a pickle file
with open('train_test_data.pkl', 'wb') as file:
    pickle.dump(train_test_data, file)
with open('your_label_encoder_file.pkl', 'wb') as label_encoder_file:
    pickle.dump(le, label_encoder_file)


In [101]:
print("Keys in the dictionary:", model_dict.keys())

Keys in the dictionary: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])


In [112]:
import os
import joblib
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13), axis=1)
    return mfccs

dataset_path = '/content/drive/MyDrive/train_emotion'

X = []
y = []
emotion_folders = ['anger', 'sad', 'fear', 'disgust', 'surprise']

for emotion_folder in emotion_folders:
    emotion_path = os.path.join(dataset_path, emotion_folder)
    for audio_file in os.listdir(emotion_path):
        audio_path = os.path.join(emotion_path, audio_file)
        features = extract_features(audio_path)
        X.append(features)
        y.append(emotion_folder)

X = np.vstack(X)
y = np.array(y)

le = LabelEncoder()
y_encoded = le.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(emotion_folders), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))



test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

model.save('emotion_model.h5')
joblib.dump(le, 'label_encoder.pkl')


Epoch 1/20
7/7 [==============================] - 3s 84ms/step - loss: 1.2797 - accuracy: 0.5181 - val_loss: 90.3651 - val_accuracy: 0.1899
Epoch 2/20
7/7 [==============================] - 0s 26ms/step - loss: 0.6187 - accuracy: 0.7807 - val_loss: 80.5978 - val_accuracy: 0.1899
Epoch 3/20
7/7 [==============================] - 0s 22ms/step - loss: 0.4267 - accuracy: 0.8627 - val_loss: 67.8536 - val_accuracy: 0.1899
Epoch 4/20
7/7 [==============================] - 0s 26ms/step - loss: 0.3284 - accuracy: 0.8964 - val_loss: 55.6400 - val_accuracy: 0.1899
Epoch 5/20
7/7 [==============================] - 0s 24ms/step - loss: 0.2528 - accuracy: 0.9036 - val_loss: 43.8348 - val_accuracy: 0.1899
Epoch 6/20
7/7 [==============================] - 0s 26ms/step - loss: 0.2696 - accuracy: 0.8867 - val_loss: 36.0425 - val_accuracy: 0.1899
Epoch 7/20
7/7 [==============================] - 0s 27ms/step - loss: 0.2131 - accuracy: 0.9277 - val_loss: 30.6230 - val_accuracy: 0.1899
Epoch 8/20
7/7 [====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['label_encoder.pkl']

In [116]:

import tensorflow as tf
loaded_model = tf.keras.models.load_model('emotion_model.h5')
loaded_le = joblib.load('label_encoder.pkl')

new_audio_file_path = '/content/mummy.m4a'

new_features = extract_features(new_audio_file_path)

new_features = new_features.reshape(1, new_features.shape[0], 1)

predictions = loaded_model.predict(new_features)
predicted_label = loaded_le.inverse_transform([np.argmax(predictions)])

print(f'Predicted Emotion: {predicted_label[0]}')


<ipython-input-112-ad2c136afbe1>:12: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 [==============================] - 0s 287ms/step
Predicted Emotion: sad
